In [19]:
from flask import Flask, jsonify
import psycopg2
import os
from dotenv import load_dotenv

In [21]:
# DB connection testing - get all data 

conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            dbname=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD')
        )
cursor = conn.cursor()
cursor.execute('SELECT * FROM fashion_product')
data = cursor.fetchall()
data

[('19',
  '1',
  'Dress',
  'Adidas',
  "Men's Fashion",
  '40',
  '1.0431592108361825',
  'Black',
  'XL'),
 ('97',
  '2',
  'Shoes',
  'H&M',
  "Women's Fashion",
  '82',
  '4.026416271141911',
  'Black',
  'L'),
 ('25',
  '3',
  'Dress',
  'Adidas',
  "Women's Fashion",
  '44',
  '3.337937559377053',
  'Yellow',
  'XL'),
 ('57',
  '4',
  'Shoes',
  'Zara',
  "Men's Fashion",
  '23',
  '1.0495229563128543',
  'White',
  'S'),
 ('79',
  '5',
  'T-shirt',
  'Adidas',
  "Men's Fashion",
  '79',
  '4.302773408398684',
  'Black',
  'M'),
 ('98',
  '6',
  'Dress',
  'Adidas',
  "Men's Fashion",
  '47',
  '1.3795657395330458',
  'Yellow',
  'L'),
 ('16',
  '7',
  'Jeans',
  'Gucci',
  "Men's Fashion",
  '37',
  '1.3567503746842564',
  'White',
  'XL'),
 ('63',
  '8',
  'Sweater',
  'Zara',
  "Kids' Fashion",
  '64',
  '4.36030328941572',
  'Blue',
  'XL'),
 ('96',
  '9',
  'Sweater',
  'H&M',
  "Men's Fashion",
  '53',
  '4.466181876278437',
  'Green',
  'XL'),
 ('36',
  '10',
  'T-shirt',


In [23]:
# Buliding API application

app = Flask(__name__)

# Context manager for PostgreSQL connection
load_dotenv()
os.getenv('DB_HOST')
class PostgreSQLConnection:
    def __init__(self):
        # Retrieve database credentials from environment variables
        self.conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            dbname=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD')
        )

    def __enter__(self):
        return self.conn.cursor()

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            self.conn.commit()
        else:
            self.conn.rollback()
        self.conn.close()

############ GET (ALL) #################

# Function to fetch data from fashion_product table
def fetch_data():
    with PostgreSQLConnection() as cursor:
        cursor.execute('SELECT * FROM fashion_product')
        col_names = [desc[0] for desc in cursor.description]
        data = cursor.fetchall()
    return col_names, data

# GET method to retrieve all data
@app.route('/api/products', methods=['GET'])
def fetch_all_product_data():
    if request.method == "GET":
        col_names, data = fetch_data()
        response_data = [{col_name: value for col_name, value in zip(col_names, row)} for row in data]
        return jsonify(response_data)

############ POST #################

# Function to insert new data into the fashion_product table
def insert_data(data):
    with PostgreSQLConnection() as cursor:
        insert_query = f'INSERT INTO fashion_product ("User ID", "Product ID", "Product Name", "Brand", "Category", "Price", "Rating", "Color", "Size") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'
        cursor.execute(insert_query, data)

# POST method to insert new data
@app.route('/api/products', methods=['POST'])
def add_product():
    try:
        data = request.get_json()
        insert_data((data.get('User ID'), data.get('Product ID'), data.get('Product Name'), data.get('Brand'), data.get('Category'), data.get('Price'), data.get('Rating'), data.get('Color'), data.get('Size')))
        return jsonify({'message': 'Data inserted successfully'}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 400

############ GET #################

# Function to fetch data for a specific product ID
def fetch_product_data(product_id):
    with PostgreSQLConnection() as cursor:
        cursor.execute('SELECT * FROM fashion_product WHERE "Product ID" = %s', (product_id,))
        col_names = [desc[0] for desc in cursor.description]
        data = cursor.fetchall()
    return col_names, data

# GET method to retrieve data for a specific product ID
@app.route('/api/products/<product_id>', methods=['GET'])
def get_product_by_id(product_id):
    try:
        col_names, data = fetch_product_data(product_id)
        if data:
            response_data = [{col_name: value for col_name, value in zip(col_names, row)} for row in data]
            return jsonify(response_data), 200
        else:
            return jsonify({'error': 'Product ID not found'}), 404
    except Exception as e:
        return jsonify({'error': str(e)}), 500


############ DELETE #################

# DELETE method to delete a specific product by its ID
@app.route('/api/products/<product_id>', methods=['DELETE'])
def delete_product(product_id):
    try:
        with PostgreSQLConnection() as cursor:
            cursor.execute('DELETE FROM fashion_product WHERE "Product ID" = %s', (product_id,))
        return jsonify({'message': f'Product with ID {product_id} deleted successfully'}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 400

############ PUT #################
    # Function to update data in the fashion_product table
def update_data(product_id, data):
    with PostgreSQLConnection() as cursor:
        update_query = '''
        UPDATE fashion_product
        SET "User ID" = %s, "Product Name" = %s, "Brand" = %s, "Category" = %s, "Price" = %s, "Rating" = %s, "Color" = %s, "Size" = %s
        WHERE "Product ID" = %s
        '''
        cursor.execute(update_query, (data.get('User ID'), data.get('Product Name'), data.get('Brand'), data.get('Category'), data.get('Price'), data.get('Rating'), data.get('Color'), data.get('Size'), product_id))

# PUT method to update existing data
@app.route('/api/products/<product_id>', methods=['PUT'])
def update_product(product_id):
    try:
        data = request.get_json()
        update_data(product_id, data)
        return jsonify({'message': f'Data for Product ID {product_id} updated successfully'}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 400



if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000


Press CTRL+C to quit
127.0.0.1 - - [21/Mar/2024 17:19:41] "GET /api/products HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 17:19:46] "POST /api/products HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 17:19:48] "GET /api/products/1001 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 17:19:50] "PUT /api/products/1001 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 17:19:52] "GET /api/products/1001 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 17:19:56] "DELETE /api/products/1001 HTTP/1.1" 200 -
127.0.0.1 - - [21/Mar/2024 17:19:58] "GET /api/products/1001 HTTP/1.1" 404 -
